In [ ]:
# !pip install ultralytics, opencv-python

In [ ]:
from ultralytics import YOLO
import cv2

#***Vehicle count for one way road***

In [ ]:
vid = cv2.VideoCapture("traffic_vid.mp4")
model = YOLO('yolov8n.pt')
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(vid.get(cv2.CAP_PROP_FPS))
codec = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('traffic_vid_result.avi', codec, fps, (1280, 720))

inx1,iny1,inx2,iny2 = 0, 450, 1280, 480
count = 0
in_ids = []
while True:

    return_value, img = vid.read()
    if return_value:
        img = cv2.resize(img, (1280, 720))
    else:
        print('Video has ended or failed, try a different video format!')
        break
    results = model.track(img,persist=True,classes=[2])
    frame= results[0]
    image = frame.orig_img
    corrs = frame.boxes.xyxy.tolist()
    if results[0].boxes.id == None:
      ids = [0]
    else:
      ids = results[0].boxes.id.cpu().numpy().astype(int)
    for cor, id in zip(corrs, ids):
        center_point = [int((int(cor[0])+int(cor[2]))/2),int((int(cor[1])+int(cor[3]))/2)]
        cv2.rectangle(image, (int(cor[0]),int(cor[1])),(int(cor[2]),int(cor[3])), (255,211,67), 2)
        cv2.rectangle(image, (int(cor[0]),int(cor[1])-30), (int(cor[0])+110,int(cor[1])), (255,211,67), -1)
        cv2.putText(image,f"#Id:{id}",(int(cor[0]),int(cor[1])-5),cv2.FONT_HERSHEY_PLAIN,2,(0, 0, 0),2)
        if center_point[1]>500:
          # print(in_ids)
          # print(id)
          if id not in in_ids:
            count += 1
            in_ids.append(id)
        cv2.line(image, (0,500),(1280,500), (255,255,255), 2)
        cv2.rectangle(image, (30,30), (320,90), (255,211,67), -1)
        cv2.circle(image,(320,60) , 30, (255,211,67), -1)
        cv2.circle(image, (30,60), 30, (255,211,67), -1)
        cv2.putText(image,'entry:'+str(count), (50,75), cv2.FONT_HERSHEY_PLAIN, 4, (0,0,0),3)

    out.write(img)
    # cv2.imshow("Image", image)
    if cv2.waitKey(25) & 0xFF == ord('q'):
            break
vid.release()
# cv2.destroyAllWindows()
out.release()

# **Vehicle count for two way road**

In [ ]:
vid = cv2.VideoCapture("vehicle-counting.mp4")
model = YOLO('yolov8n.pt')
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(vid.get(cv2.CAP_PROP_FPS))
codec = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('vehicle-count_result.avi', codec, fps, (width, height))

# inx1,iny1,inx2,iny2 = 0, 450, 1280, 480
count = 0
count_out = 0
in_ids = []
out_ids = []
while True:

    return_value, img = vid.read()
    if return_value:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         frame_num += 1
        img = cv2.resize(img, (width, height))
    else:
        print('Video has ended or failed, try a different video format!')
        break
    results = model.track(img,persist=True,classes=[2,7])
    frame= results[0]
    image = frame.orig_img
    corrs = frame.boxes.xyxy.tolist()
    # boxes = frame.boxes.xyxy.tolist()
    if results[0].boxes.id == None:
      ids = [0]
    else:
      ids = results[0].boxes.id.cpu().numpy().astype(int)
    for cor, id in zip(corrs, ids):
        center_point = [int((int(cor[0])+int(cor[2]))/2),int((int(cor[1])+int(cor[3]))/2)]
        cv2.rectangle(image, (int(cor[0]),int(cor[1])),(int(cor[2]),int(cor[3])), (255,211,67), 2)
        cv2.rectangle(image, (int(cor[0]),int(cor[1])-30), (int(cor[0])+110,int(cor[1])), (255,211,67), -1)
        cv2.putText(image,f"#Id:{id}",(int(cor[0]),int(cor[1])-5),cv2.FONT_HERSHEY_PLAIN,2,(0, 0, 0),2)
        if center_point[1]>1500 and center_point[0]<2000:
          # print(in_ids)
          # print(id)
          if id not in in_ids:
            count += 1
            in_ids.append(id)
        cv2.line(image, (0,1500),(3840,1500), (255,255,255), 2)

        if center_point[1]<1500 and center_point[0]>2000:
          # print(in_ids)
          # print(id)
          if id not in out_ids:
            count_out += 1
            out_ids.append(id)

        cv2.rectangle(image, (100,50), (600,150), (100,255,10), -1)
        cv2.circle(image,(600,100) , 50, (100,255,10), -1)
        cv2.circle(image, (100,100), 50, (100,255,10), -1)
        cv2.putText(image,'entry: '+str(count), (100,125), cv2.FONT_HERSHEY_PLAIN, 6, (0,0,0),3)

        cv2.rectangle(image, (100,200), (600,300), (255,30,10), -1)
        cv2.circle(image,(600,250) , 50, (255,30,10), -1)
        cv2.circle(image, (100,250), 50, (255,30,10), -1)
        cv2.putText(image,'exit: '+str(count_out), (100,275), cv2.FONT_HERSHEY_PLAIN, 6, (0,0,0),3)

    out.write(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # cv2.imshow("Image", image)
    if cv2.waitKey(25) & 0xFF == ord('q'):
            break
vid.release()
# cv2.destroyAllWindows()
out.release()